In [160]:
import os
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import re
from rapidfuzz import process, fuzz
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment
from datetime import datetime
from openpyxl.utils.dataframe import dataframe_to_rows


# Build engine
DATABASE_DSN = (
    "postgresql://postgres.avcznjglmqhmzqtsrlfg:Czheyuan0227@"
    "aws-0-us-east-2.pooler.supabase.com:6543/postgres?sslmode=require"
)
engine = create_engine(DATABASE_DSN, pool_pre_ping=True)



#<---Formatting Lookup Table--->
def fmt_date(x):
    # Works whether x is Timestamp, date, or str/NaN
    try:
        if pd.isna(x): return ''
        return pd.to_datetime(x).strftime('%Y-%m-%d')
    except Exception:
        return str(x)





#<---Item Listing Fuzyy Search--->
# Load CSV
Itemlisting = pd.read_csv("Item Listing.csv", dtype=str, encoding_errors="ignore").fillna("")
# Keep only the text after the first ":" in Item
Itemlisting["Item"] = Itemlisting["Item"].str.split(":", n=1).str[-1].str.strip()

Itemlisting.columns = Itemlisting.columns.str.strip()

def normalize(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9]", "", str(s)).lower()

# Normalized item column for searching
Itemlisting["__item_norm"] = Itemlisting["Item"].map(normalize)

def search_items(query: str, top_k: int = 15) -> pd.DataFrame:
    q = normalize(query)
    if not q:
        return pd.DataFrame(columns=Itemlisting.columns)

    # 1) Substring matches
    # Split query into words
    words = q.split()

    # Require all words to appear
    mask = Itemlisting["__item_norm"].apply(lambda name: all(w in name for w in words))
    direct = Itemlisting.loc[mask, Itemlisting.columns].copy()


    # 2) Fuzzy fallback 
    matches = process.extract(query, Itemlisting["Item"].tolist(), scorer=fuzz.WRatio, limit=top_k*2)
    fuzzy_rows = []
    seen = set()
    for _text, _score, idx in matches:
        if idx not in seen:
            seen.add(idx)
            fuzzy_rows.append(Itemlisting.iloc[idx])
    fuzzy = pd.DataFrame(fuzzy_rows)

    # 3) Combine results and dedupe by Item
    out = pd.concat([direct, fuzzy], ignore_index=True).drop_duplicates(subset=["Item"])
    return out.head(top_k)[["Item", "Description", 'Type', 'Cost', 'Price', 'Preferred Vendor']]


In [136]:
df.columns

Index(['SO Entry Date', 'Customer', 'Customer PO', 'WO', 'Product Number',
       'Qty', 'Lead Time', 'Picked', 'Part_Number', 'Reorder Pt (Min)', 'Max',
       'On Hand', 'On Sales Order', 'Stock_Available', 'Order', 'On PO',
       'Reorder Qty', 'Next Deliv', 'Sales/Week', 'Picked_Qty',
       'Assigned Q'ty', 'In Stock(Inventory)', 'Qty(+)',
       'Available + Pre-installed PO', 'Recommended Restock Qty',
       'Component_Status'],
      dtype='object')

In [142]:

# Read entire table
df = pd.read_sql_table("wo_structured", con=engine, schema="public")

ERP_df= df[['SO Entry Date', 'Customer', 'Customer PO', 'WO', 'Product Number', 'Qty', 
                              "Available + Pre-installed PO", 'Stock_Available', "Assigned Q'ty", 'In Stock(Inventory)', 'On Hand', 'On Sales Order', 'On PO', 'Reorder Pt (Min)', 'Sales/Week', 'Recommended Restock Qty', 'Lead Time', 'Picked']]

ERP_df = ERP_df.copy()

ERP_df['SO_Status'] = np.where(
    ERP_df['Stock_Available'] < 0, 'Shortage', 'Available'
)

# --- REORDER COLUMNS (do this BEFORE saving to Excel) ---
desired_order = [
    "SO Entry Date", "Customer", "Customer PO", "WO", "Product Number", "Qty",
    "Available + Pre-installed PO", "Stock_Available", 'On Hand', "In Stock(Inventory)",
    "Assigned Q'ty", "On Sales Order", "On PO",
    "Reorder Pt (Min)", 'Sales/Week', "Recommended Restock Qty", 
    'Lead Time', "Picked", "SO_Status", 
]

# keep only existing columns in that order, then append any remaining columns
ordered = [c for c in desired_order if c in ERP_df.columns] + \
          [c for c in ERP_df.columns if c not in desired_order]

ERP_df = ERP_df.reindex(columns=ordered)


filtered_df = ERP_df[ERP_df['WO'] == 'SO-20251306']

# filtered_df = ERP_df[ERP_df['Customer'] == 'Aerflite Canada Inc.']


# Make Lead Time real datetimes
filtered_df = filtered_df.copy()
filtered_df['Lead Time'] = pd.to_datetime(filtered_df['Lead Time'], errors='coerce')


# Create styled object
styled = (
    filtered_df.style
    .map(lambda v: 'background-color: red; color: white' if v == 'Shortage' else '', subset='SO_Status')
    # General table & header styling
    .set_table_styles([
        # Header style
        {'selector': 'thead th',
         'props': [
             ('background-color', '#444'),
             ('color', 'white'),
             ('font-weight', 'bold'),
             ('text-align', 'center'),
             ('border', '1px solid #777'),
             ('padding', '6px 8px')
         ]},

        # Table body cells
        {'selector': 'td',
         'props': [
             ('color', 'white'),
             ('border', '1px solid #555'),
             ('text-align', 'center'),
             ('padding', '4px 6px')
         ]},

        # Highlight column header (Available + Pre-installed PO)
        {'selector': 'th.col' + str(filtered_df.columns.get_loc('Available + Pre-installed PO')),
         'props': [
             ('background-color', '#ffae42'),
             ('color', 'black'),
             ('font-weight', 'bold'),
             ('border', '1px solid #777'),
             ('min-width', '100px')
         ]},

        # Widen Lead Time column
        {'selector': 'th.col' + str(filtered_df.columns.get_loc('Lead Time')),
         'props': [
             ('min-width', '120px'),     # increase column width
             ('max-width', '150px')
         ]},

        # Optional — hover effect
        {'selector': 'tbody tr:hover',
         'props': [('background-color', '#333')]}
    ])
    .format({'Lead Time': fmt_date}, precision=0, na_rep='', thousands=',')
)

display(styled)




,SO Entry Date,Customer,Customer PO,WO,Product Number,Qty,Available + Pre-installed PO,Stock_Available,On Hand,In Stock(Inventory),Assigned Q'ty,On Sales Order,On PO,Reorder Pt (Min),Sales/Week,Recommended Restock Qty,Lead Time,Picked,SO_Status
575,09/12/2025,ADVANCED TECH & RESEARCH,PO-036318,SO-20251306,Nuvo-10003,3,-3,-3,0,0,3,3,3,0,0,0,2025-10-24,No,Shortage
576,09/12/2025,ADVANCED TECH & RESEARCH,PO-036318,SO-20251306,i7-12700,3,0,0,12,11,12,12,0,0,1,0,2025-10-24,No,Available
577,09/12/2025,ADVANCED TECH & RESEARCH,PO-036318,SO-20251306,DDR5-32GB-48-SM,3,68,68,104,104,13,36,0,50,7,0,2025-10-24,No,Available
578,09/12/2025,ADVANCED TECH & RESEARCH,PO-036318,SO-20251306,M.280-SSD-2TB-SATA-TLC5ET-TD,3,0,0,3,3,3,3,0,2,0,2,2025-10-24,No,Available
579,09/12/2025,ADVANCED TECH & RESEARCH,PO-036318,SO-20251306,mPCIeHS-WiFi-121N,3,8,8,52,42,34,44,0,15,3,7,2025-10-24,No,Available
580,09/12/2025,ADVANCED TECH & RESEARCH,PO-036318,SO-20251306,Extnd-mPCIeHS,3,14,14,77,67,51,63,10,50,6,26,2025-10-24,No,Available
581,09/12/2025,ADVANCED TECH & RESEARCH,PO-036318,SO-20251306,Win11IoT24-High,3,5,5,11,11,6,6,0,0,1,0,2025-10-24,No,Available
582,09/12/2025,ADVANCED TECH & RESEARCH,PO-036318,SO-20251306,Ant-RP_SMAM-WiFi-196MM1,6,-18,-228,157,89,358,385,262,50,13,16,2025-10-24,No,Shortage
583,09/12/2025,ADVANCED TECH & RESEARCH,PO-036318,SO-20251306,Cbl-MHF-RP_SMAF-30CM,6,0,0,112,92,90,112,25,50,10,25,2025-10-24,No,Available


In [116]:
filtered_df = ERP_df[ERP_df['Product Number'] == 'DDR5-32GB-48WT-SM'].sort_values(by='Lead Time',ascending=True)
filtered_df

,SO Entry Date,Customer,Customer PO,WO,Product Number,Qty,Available + Pre-installed PO,Stock_Available,On Hand,In Stock(Inventory),ATP(LT),On Sales Order,On PO,Reorder Pt (Min),Recommended Restock Qty,Lead Time,Picked,SO_Status
242,08/12/2025,Near Earth Autonomy,7274,SO-20251145,DDR5-32GB-48WT-SM,2.0,108.0,90.0,126.0,114.0,92.0,36.0,18.0,50.0,0.0,2025-10-15,No,Available
416,08/28/2025,"US Army Corps of Engineers, ERDC-IEI",QTD_USACE_2507093,SO-20251236,DDR5-32GB-48WT-SM,4.0,108.0,90.0,126.0,114.0,92.0,36.0,18.0,50.0,0.0,2025-10-21,No,Available
489,09/05/2025,Université Laval,0001513611,SO-20251261,DDR5-32GB-48WT-SM,2.0,108.0,90.0,126.0,114.0,92.0,36.0,18.0,50.0,0.0,2025-10-24,No,Available
147,06/06/2025,"Applied Intuition, Inc.",AI-001279,SO-20250800,DDR5-32GB-48WT-SM,4.0,108.0,90.0,126.0,114.0,92.0,36.0,18.0,50.0,0.0,2025-10-31,Picked,Available
166,07/15/2025,"Applied Intuition, Inc.","AI-001620, AI-001621",SO-20250995,DDR5-32GB-48WT-SM,8.0,108.0,90.0,126.0,114.0,92.0,36.0,18.0,50.0,0.0,2025-10-31,Picked,Available
427,09/03/2025,Carnegie Robotics LLC,PO-0138392,SO-20251248,DDR5-32GB-48WT-SM,6.0,108.0,90.0,126.0,114.0,92.0,36.0,18.0,50.0,0.0,2025-10-31,No,Available
292,08/19/2025,Near Earth Autonomy,7291,SO-20251174,DDR5-32GB-48WT-SM,4.0,108.0,90.0,126.0,114.0,92.0,36.0,18.0,50.0,0.0,2025-11-07,No,Available
722,09/26/2025,ASRC Federal,L054,SO-20251365,DDR5-32GB-48WT-SM,4.0,108.0,90.0,126.0,114.0,92.0,36.0,18.0,50.0,0.0,2025-11-19,No,Available
684,09/23/2025,FEV North America Inc.,1034004597,SO-20251348,DDR5-32GB-48WT-SM,2.0,108.0,90.0,126.0,114.0,92.0,36.0,18.0,50.0,0.0,2025-12-31,No,Available


## Part Name Lookup

In [40]:
results = search_items("cblm12")
print(results.to_string(index=False))

                           Item                                                                                         Description           Type  Cost Price           Preferred Vendor
         Cbl-M12A10M-DB9M-180CM Cbl-M12A10M-DB9M-180CM..M12 A-Code 10P (male) to DB9 (male), Length: 180CM. Industrial computer ... Inventory Part 14.00 25.00 Neousys Technology Incorp.
    Cbl-M12A17M-2DB9M_OW2-180CM Cbl-M12A17M-2DB9M_OW2-180CM..M12 (17-pole-A-coded) to 2xDB9 (Male) and 1xopen wire 2P, Length: 1... Inventory Part 15.00 19.00 Neousys Technology Incorp.
         Cbl-M12A17M-VGA-180CM1 Cbl-M12A17M-VGA-180CM1..M12 (17-pole-A-coded) to VGA (male), Length : 180CM. Industrial computer... Inventory Part 16.00 20.00 Neousys Technology Incorp.
         Cbl-M12A17M-VGA-180CM2                        Cbl-M12A17M-VGA-180CM2..M12 (17-pole-A-coded) to VGA (male), Length : 180CM. Inventory Part 10.00 13.00 Neousys Technology Incorp.
         Cbl-M12A17M-VGA-180CM3 Cbl-M12A17M-VGA-180CM3..M12 (17-pole-A

## Items that can still be assigned although have negative available

In [119]:
filtered_df1 = ERP_df[(ERP_df['Stock_Available'] < 0) & (ERP_df['Available + Pre-installed PO'] > 0)]
unique_parts_df = filtered_df1.drop_duplicates(subset=['Product Number'])
unique_parts_df[['Product Number', 'Stock_Available', 'On PO', 'Available + Pre-installed PO']]

,Product Number,Stock_Available,On PO,Available + Pre-installed PO
76,M.2-LTE-7455,-80.0,105.0,25.0
77,Risr-M2B-mPCIe-SIMslot,-103.0,115.0,2.0
78,Cbl-MHF4-SMAF-30CM,-295.0,340.0,20.0
140,i7-9700E,-14.0,30.0,6.0
148,M.280-SSD-2TB-PCIe44-TLC5WT-TD,-19.0,39.0,12.0
154,GC-Jetson-AGX32GB-Orin-Nvidia,-103.0,105.0,2.0
159,NRU-230V-AWP-JAO64G,-23.0,25.0,2.0
160,M.280-SSD-2TB-PCIe44-TLC5ET-TD,-11.0,31.0,15.0
169,POC-330,-25.0,50.0,25.0
170,DDR3L-4GB-WT18-DL1,-27.0,50.0,23.0


## SO not has been assigned LT yet

In [163]:
# --- Your filter ---
assigned_mask = (
    (ERP_df["Lead Time"].dt.month.eq(7)  & ERP_df["Lead Time"].dt.day.eq(4)) |
    (ERP_df["Lead Time"].dt.month.eq(12) & ERP_df["Lead Time"].dt.day.eq(31))
)
Not_assgned_SO = ERP_df[assigned_mask].copy()

# --- Output path ---
output_path = r"Not_assigned_SO.xlsx"

# =======================
# 1) SAVE SHEET1 (ALL ROWS)
# =======================
# Find current first sheet's name, then replace just that sheet
_wb = load_workbook(output_path)
first_sheet_name = _wb.sheetnames[0]
_wb.close()

with pd.ExcelWriter(output_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    Not_assgned_SO.to_excel(writer, sheet_name=first_sheet_name, index=False)

# ======================= 
# 2) OPEN WORKBOOK & PREP STYLING
# =======================
from datetime import datetime
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl import load_workbook

wb = load_workbook(output_path)
ws = wb.worksheets[0] 

# Freeze the first row (header)
ws.freeze_panes = "A2"

# Build header -> column index map for Sheet1
wo_col = None
status_col = None
col_map = {}
for idx, cell in enumerate(ws[1], 1):  # header row
    col_map[cell.value] = idx
    if cell.value == "WO":
        wo_col = idx
    if cell.value == "SO_Status":
        status_col = idx

# Define fills & fonts
gray_fill   = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")
white_fill  = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")  # for Recommended Restock Qty > 0
red_font    = Font(color="FF0000")
center_align = Alignment(horizontal="center", vertical="center")

# =======================
# 3) FORMAT SHEET1 (band by WO, red rows for Shortage, widths, alignment)
# =======================
current_wo = None
fill_toggle = False

if wo_col is not None and status_col is not None:
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        wo_cell = row[wo_col - 1]
        status_cell = row[status_col - 1]

        # Toggle background when WO changes
        if wo_cell.value != current_wo:
            current_wo = wo_cell.value
            fill_toggle = not fill_toggle

        # Apply background fill to row
        fill = gray_fill if fill_toggle else white_fill
        for c in row:
            c.fill = fill

        # If shortage, make entire row red font
        if status_cell.value == "Shortage":
            for c in row:
                c.font = red_font

# NEW: Highlight "Recommended Restock Qty" if > 0
target_col_name = "Recommended Restock Qty"
if target_col_name in col_map:
    col_idx = col_map[target_col_name]
    for (cell,) in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=col_idx, max_col=col_idx):
        # Safely coerce to number
        try:
            val = float(cell.value)
        except (TypeError, ValueError):
            val = 0
        if val > 0:
            cell.fill = yellow_fill  # override banding fill for this cell only

# Set column widths (if they exist)
col_widths = {
    'SO Entry Date': 15,
    "Product Number": 30,
    "Customer": 25,
    "Customer PO": 15,
    "WO": 15,
    "Qty": 10,
    "Stock_Available": 15,
    'Available + Pre-installed PO': 25,
    'In Stock(Inventory)': 20,
    'Reorder Pt (Min)': 15,
    'Recommended Restock Qty': 20,
    'On Sales Order': 15
}
for col_name, width in col_widths.items():
    if col_name in col_map:
        col_letter = ws.cell(row=1, column=col_map[col_name]).column_letter
        ws.column_dimensions[col_letter].width = width

# Center-align specific columns on Sheet1
for col_name in ["Qty", "Available + Pre-installed PO", "Stock_Available"]:
    if col_name in col_map:
        col_idx = col_map[col_name]
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=col_idx, max_col=col_idx):
            for cell in row:
                cell.alignment = center_align

# Rename Sheet1 to today's date (e.g., "2025-10-05")
today_str = datetime.today().strftime("%Y-%m-%d")
ws.title = today_str

# # =======================
# # 4) CREATE SHEET2 (SO Entry Date == 2025-10-03) + SAME FORMATTING
# # =======================
# # Filter the DataFrame for that specific date
# # target_date = datetime(2025, 10, 3).date()
# target_date = datetime.today().date()
# mask_target = pd.to_datetime(Not_assgned_SO["SO Entry Date"], errors="coerce").dt.date.eq(target_date)
# today_only = Not_assgned_SO.loc[mask_target].copy()

# # Re-create Sheet2
# if "Sheet2" in wb.sheetnames:
#     wb.remove(wb["Sheet2"])
# wa = wb.create_sheet("Sheet2")

# # Write DataFrame rows into Sheet2
# for r in dataframe_to_rows(today_only, index=False, header=True):
#     wa.append(r)

# # Build header map for Sheet2
# wo_col2 = None
# status_col2 = None
# col_map2 = {}
# if wa.max_row >= 1:
#     for idx, cell in enumerate(wa[1], 1):  # header row
#         col_map2[cell.value] = idx
#         if cell.value == "WO":
#             wo_col2 = idx
#         if cell.value == "SO_Status":
#             status_col2 = idx

# # Apply the SAME banding-by-WO and shortage red-font to Sheet2
# current_wo2 = None
# fill_toggle2 = False

# if wo_col2 is not None and status_col2 is not None:
#     for row in wa.iter_rows(min_row=2, max_row=wa.max_row):
#         wo_cell2 = row[wo_col2 - 1]
#         status_cell2 = row[status_col2 - 1]

#         if wo_cell2.value != current_wo2:
#             current_wo2 = wo_cell2.value
#             fill_toggle2 = not fill_toggle2

#         fill2 = gray_fill if fill_toggle2 else white_fill
#         for c in row:
#             c.fill = fill2

#         if status_cell2.value == "Shortage":
#             for c in row:
#                 c.font = red_font

# # Set the SAME column widths on Sheet2
# for col_name, width in col_widths.items():
#     if col_name in col_map2:
#         col_letter2 = wa.cell(row=1, column=col_map2[col_name]).column_letter
#         wa.column_dimensions[col_letter2].width = width

# # Center-align SAME columns on Sheet2
# for col_name in ["Qty", "Available + installed PO", "Stock_Available"]:
#     if col_name in col_map2:
#         col_idx2 = col_map2[col_name]
#         for row in wa.iter_rows(min_row=2, max_row=wa.max_row, min_col=col_idx2, max_col=col_idx2):
#             for cell in row:
#                 cell.alignment = center_align

# =======================
# 5) SAVE + SUMMARY
# =======================
wb.save(output_path)

unique_wo_count = Not_assgned_SO["WO"].nunique() if "WO" in Not_assgned_SO.columns else 0
print(f"Number of unassigned WOs: {unique_wo_count}")
# print(f"Written:\n  - Sheet '{today_str}' (formerly Sheet1)\n  - Sheet 'Sheet2' (SO Entry Date == {target_date}) -> {len(today_only)} rows")


Number of unassigned WOs: 31


## Vendor POD

In [153]:
NT_Shipping = pd.read_excel(r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\Daily Update\NTA_Shipping schedule_20251007.xlsx")
NT_Shipping.loc[NT_Shipping['Customer PO No.'] == 'POD-251405']

,Date,SO NO.,Ship to,Customer PO No.,Reference,Project Code,Model Name,Assemble Option,Qty,Remark,Ship Date,Description
394,2025-10-02,SO25100034,"Neousys Technology America, Inc.",POD-251405,NTA_INVENTORY,NaN,Nuvo-7006LP,No,6,NaN,TBC,Intel® Coffee Lake 8th-Gen Core™ i fanless em...
395,2025-10-02,SO25100034,"Neousys Technology America, Inc.",POD-251405,NTA_INVENTORY,NaN,Nuvo-9160GC-PoE,No,4,NaN,TBC,Ruggedized GPU-Computing Platform Supporting 1...
396,2025-10-02,SO25100034,"Neousys Technology America, Inc.",POD-251405,NTA_INVENTORY,NaN,mPCIeHS-BTWifi-WT-6218,No,10,NaN,TBC,"802.11ac/a/b/g/n, 2T2R WiFi + BT4.2 half size ..."
397,2025-10-02,SO25100034,"Neousys Technology America, Inc.",POD-251405,NTA_INVENTORY,NaN,Ant-RP_SMAM-WiFi-196MM1,No,50,NaN,TBC,"WiFi Antenna, Swivel type, Frequency: 2.4/5.8 ..."
398,2025-10-02,SO25100034,"Neousys Technology America, Inc.",POD-251405,NTA_INVENTORY,NaN,Cbl-MHF-SMAF-30CM1,No,2,NaN,TBC,"LTE Internal Cable, I-PEX MHF(Female) to SMA (..."
399,2025-10-02,SO25100034,"Neousys Technology America, Inc.",POD-251405,NTA_INVENTORY,NaN,Gpubr-Nuvo9160-01,No,20,NaN,TBC,"Nuvo-9160GC GPU bracket kit for RTX A2000, RTX..."
400,2025-10-02,SO25100034,"Neousys Technology America, Inc.",POD-251405,NTA_INVENTORY,NaN,M.280-SSD-1TB-PCIe44-TLC5-PN,No,10,NaN,TBC,"M.2 SSD 1TB, Key M 2280, TLC, NVMe PCIe Gen 4x..."
401,2025-10-02,SO25100034,"Neousys Technology America, Inc.",POD-251405,NTA_INVENTORY,NaN,Extnd-mPCIeHS,No,10,NaN,TBC,Sparklan Half Mini-PCIe Extender
402,2025-10-02,SO25100034,"Neousys Technology America, Inc.",POD-251405,NTA_INVENTORY,NaN,Risr-M2B-mPCIe-SIMslot,No,10,NaN,TBC,NGFF M.2 key B to mini-PCIe adapter include SI...
403,2025-10-02,SO25100034,"Neousys Technology America, Inc.",POD-251405,NTA_INVENTORY,NaN,Cbl-MHF4-SMAF-30CM,No,25,NaN,TBC,"LTE Internal Cable, IPEX MHF4(Female) to SMA(F..."


In [164]:
NT_Shipping.loc[NT_Shipping['Model Name'] == 'GC-RTX2000Ada-Nvidia']

,Date,SO NO.,Ship to,Customer PO No.,Reference,Project Code,Model Name,Assemble Option,Qty,Remark,Ship Date,Description
387,2025-09-30,SO25090519,"Neousys Technology America, Inc.",POD-251380,NTA_MONTERIS_SO-20251332,NaN,GC-RTX2000Ada-Nvidia,No,31,待採回,TBC,"NVIDIA RTX 2000 Ada Generation Graphics Card,..."


In [156]:
filtered_df = ERP_df[ERP_df['Product Number'] == 'Gpubr-Nuvo9160-01'].sort_values(by='Lead Time',ascending=True)
filtered_df

,SO Entry Date,Customer,Customer PO,WO,Product Number,Qty,Available + Pre-installed PO,Stock_Available,On Hand,In Stock(Inventory),Assigned Q'ty,On Sales Order,On PO,Reorder Pt (Min),Sales/Week,Recommended Restock Qty,Lead Time,Picked,SO_Status
405,08/28/2025,CYNGN INC.,CYNGN-1275,SO-20251229,Gpubr-Nuvo9160-01,4.0,-15.0,-15.0,3.0,3.0,4.0,18.0,30.0,5.0,3.7,0.0,2025-10-21,No,Shortage
645,09/19/2025,Monteris Medical,08000,SO-20251333,Gpubr-Nuvo9160-01,5.0,-15.0,-15.0,3.0,3.0,4.0,18.0,30.0,5.0,3.7,0.0,2025-12-31,No,Shortage
780,09/30/2025,Palladyne AI,5905,SO-20251384,Gpubr-Nuvo9160-01,8.0,-15.0,-15.0,3.0,3.0,4.0,18.0,30.0,5.0,3.7,0.0,2025-12-31,No,Shortage
908,09/30/2025,F.K. MACHINERY LTD.,4017695,SO-20251400,Gpubr-Nuvo9160-01,1.0,-15.0,-15.0,3.0,3.0,4.0,18.0,30.0,5.0,3.7,0.0,2025-12-31,No,Shortage


In [155]:
pod = df = pd.read_sql_table("Open_Purchase_Orders", con=engine, schema="public")
pod.loc[pod['Item'] == 'Gpubr-Nuvo9160-01']

,Order Date,QB Num,Name,Source Name,Item,Deliv Date,Qty(+)
75,2025/08/29,POD-251248,Neousys Technology Incorp.,Neousys Technology Incorp.,Gpubr-Nuvo9160-01,2025/09/10,5.0
76,2025/09/22,POD-251351,Neousys Technology Incorp.,Neousys Technology Incorp.,Gpubr-Nuvo9160-01,2025/10/16,5.0
77,2025/10/02,POD-251405,Neousys Technology Incorp.,Neousys Technology Incorp.,Gpubr-Nuvo9160-01,2025/10/22,20.0


In [96]:
pod.loc[pod['QB Num'] == 'Adapter-Active-DP-HDMI-BENFEI']

,Order Date,QB Num,Name,Source Name,Item,Deliv Date,Qty(+)


In [13]:
# Filter rows where Name is not 'Neousys Technology Incorp.'
filtered = pod[pod['Name'] != 'Neousys Technology Incorp.']

# Group by Item and sum Qty(+)
result = (
    filtered.groupby('Item', as_index=False)['Qty(+)']
    .sum()
)

lookup = (
    result[['Item', 'Qty(+)']]
    .drop_duplicates(subset=['Item'])         # ensures uniqueness
    .set_index('Item')['Qty(+)'] # Series: index = part_number
)

# 3) Map onto df (row count stays the same)
ERP_df['Qty(+)'] = ERP_df['Product Number'].map(lookup)
ERP_df



,Customer,Customer PO,WO,Product Number,Qty,Available + installed PO,Stock_Available,ATP(LT),In Stock(Inventory),On Hand,On Sales Order,On PO,Lead Time,Picked,SO_Status,Qty(+)
0,"CoastIPC, Inc.",P96695,EO-20250002,RGS-8805GC,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-12-31,No,Available,NaN
1,Xanthon LLC,X110992,SO-20240315,POC-715,1.0,30.0,30.0,36.0,39.0,42.0,12.0,0.0,2025-12-19,No,Available,NaN
2,Xanthon LLC,X110992,SO-20240315,DDR5-16GB-WT48-IK,1.0,47.0,47.0,47.0,138.0,182.0,135.0,0.0,2025-12-19,No,Available,NaN
3,Xanthon LLC,X110992,SO-20240315,M.280-SSD-1TB-PCIe4-TLCWT-IK1,1.0,51.0,51.0,51.0,52.0,52.0,1.0,0.0,2025-12-19,No,Available,NaN
4,Xanthon LLC,X110992,SO-20240315,AccsyBx-FAN-POC-700,1.0,69.0,69.0,69.0,70.0,70.0,1.0,0.0,2025-12-19,No,Available,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,Noah Medical U.S.,PO209761,SO-20251423,Extnd-mPCIeHS,4.0,28.0,28.0,26.0,67.0,84.0,56.0,0.0,2025-12-31,No,Available,NaN
1042,Noah Medical U.S.,PO209761,SO-20251423,Cbl-MHF-RP_SMAF-30CM,8.0,26.0,26.0,22.0,92.0,118.0,92.0,25.0,2025-12-31,No,Available,NaN
1043,Noah Medical U.S.,PO209761,SO-20251423,Ant-RP_SMAM-WiFi-196MM1,8.0,13.0,-237.0,0.0,49.0,123.0,360.0,252.0,2025-12-31,No,Shortage,250.0
1044,Noah Medical U.S.,PO209761,SO-20251424,GC-RTXA4500-PNY,10.0,6.0,6.0,37.0,37.0,37.0,31.0,0.0,2025-12-31,No,Available,NaN


### Incoming QTY (assigned to WO) + Stock_Available <= 0

In [14]:
filtered_df1 = ERP_df[(ERP_df['Qty(+)']+ERP_df['Stock_Available']) <= 0].drop_duplicates(subset=['Product Number']).drop(['Customer', 'Customer PO', 'WO'], axis=1)
filtered_df1

,Product Number,Qty,Available + installed PO,Stock_Available,ATP(LT),In Stock(Inventory),On Hand,On Sales Order,On PO,Lead Time,Picked,SO_Status,Qty(+)
5,E-2278GE,5.0,-5.0,-6.0,0.0,19.0,29.0,35.0,6.0,2025-10-24,Picked,Shortage,1.0
98,M.280-SSD-1TB-PCIe44-TLC5-PN,10.0,-11.0,-24.0,0.0,29.0,39.0,63.0,43.0,2025-10-03,Picked,Shortage,13.0
99,Cbl-M12S4F-OW4-180CM1,10.0,0.0,-40.0,0.0,0.0,10.0,50.0,40.0,2025-10-03,Picked,Shortage,40.0
143,i7-9700E,20.0,-1.0,-21.0,0.0,-1.0,19.0,40.0,30.0,2025-10-01,Picked,Shortage,20.0
156,NRU-230V-AWP,2.0,0.0,-16.0,0.0,3.0,5.0,21.0,20.0,2025-08-06,Picked,Shortage,16.0
160,AC-IMX490-H120,1.0,0.0,-1.0,0.0,0.0,1.0,2.0,1.0,2025-08-06,Picked,Shortage,1.0
161,Cbl-M12A8M-2U2TA-180CM2,3.0,0.0,-4.0,0.0,0.0,3.0,7.0,4.0,2025-08-06,Picked,Shortage,4.0
165,SSD-2TB-TLC-IK,25.0,0.0,-50.0,0.0,25.0,25.0,75.0,50.0,2025-10-08,No,Shortage,50.0
167,Cblkit-BP-NRU-230V-AWP,25.0,0.0,-53.0,0.0,25.0,26.0,79.0,57.0,2025-10-08,No,Shortage,53.0
179,Ant-RP_SMAM-WiFi-108MM,8.0,0.0,-20.0,0.0,14.0,22.0,42.0,40.0,2025-09-11,Picked,Shortage,20.0


## Ready to Pick (Ins Stock > Required Qty)

In [ ]:
# # True for WOs where ALL lines have In Stock > 0 and not been picked
# wo_ok = (
#     ERP_df.groupby("WO")
#     .apply(lambda g: (g["In Stock(Inventory)"] >= g['Qty']).all() and (g["Picked"].eq("No").all()))
# )


# # 1) list of ready WOs
# ready_wos = wo_ok[wo_ok].index.tolist()


# # 2) full rows for those WOs
# ready_df = ERP_df[ERP_df["WO"].isin(ready_wos)].copy()

# print("Ready WOs:", ready_wos)


# ready_df.to_excel(r"C:\Users\Admin\OneDrive - neousys-tech\Desktop\Output.xlsx", sheet_name="Sheet2", index=False)

Ready WOs: ['SO-20240315', 'SO-20250701', 'SO-20250726', 'SO-20250728', 'SO-20250936', 'SO-20251044', 'SO-20251111', 'SO-20251113', 'SO-20251126', 'SO-20251250', 'SO-20251280', 'SO-20251281', 'SO-20251283', 'SO-20251324', 'SO-20251338', 'SO-20251376', 'SO-20251377', 'SO-20251378', 'SO-20251391', 'SO-20251393', 'SO-20251394', 'SO-20251395', 'SO-20251419', 'SO-20251424']


/var/folders/5l/3_4dbl_x6t9cv8nl8tv33_800000gn/T/ipykernel_72071/3891689377.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g["In Stock(Inventory)"] >= g['Qty']).all() and (g["Picked"].eq("No").all()))
